# PyZX interoperability: a simple graph

In [ ]:
import os
import sys
import pyzx as zx
from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister
import qiskit.qasm2 as qasm2

root_folder = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
if root_folder not in sys.path:
    sys.path.insert(0, root_folder)

from scripts.runner import runner
from utils.interop_pyzx import pyzx_g_to_simple_g
from utils.utils_zx_graphs import kind_to_zx_type

zx.settings.colors = zx.rgb_colors
%matplotlib widget

This notebook offers an example of how to use ***topologiq*** to perform an algorithmic lattice surgery on QASM circuits. All circuits correspond to different versions of a Steane code.

## Qiskit -> QASM

Let's start by using Qiskit to produce a QASM string that can undergo algorithmic lattice surgery using ***topologiq***. 

The first step is to create the Qiskit circuit. 

In [ ]:
# Thanks [Yilun Zhao](https://github.com/Zhaoyilunnn) for the code and rationale in this block.
def steane_encoding() -> QuantumCircuit:
    qc = QuantumCircuit(10)

    qc.h(0)
    qc.cx(0, 3)
    qc.cx(0, 4)
    qc.cx(0, 5)
    qc.cx(0, 6)
    qc.h(0)

    qc.h(1)
    qc.cx(1, 3)
    qc.cx(1, 4)
    qc.cx(1, 7)
    qc.cx(1, 8)
    qc.h(1)

    qc.h(2)
    qc.cx(2, 3)
    qc.cx(2, 5)
    qc.cx(2, 7)
    qc.cx(2, 9)
    qc.h(2)

    return qc

circuit = steane_encoding()
print(circuit)

The circuit can then be converted into a PyZX circuit, which, in turn, can be converted into a PyZX graph.

In [ ]:
# Thanks [Yilun Zhao](https://github.com/Zhaoyilunnn) for the code and rationale in this block.

def qiskit2zx(circuit: QuantumCircuit) -> zx.circuit.Circuit:
    qasm_str = qasm2.dumps(circuit)
    zx_circuit = zx.circuit.Circuit.from_qasm(qasm_str)
    return zx_circuit

zx_circuit = qiskit2zx(circuit)
print(zx_circuit.gates)

zx_graph = zx_circuit.to_graph()

zx.draw(zx_graph, labels=True)  # PyZX produces Jupyter visualisations using D3.
fig_data = zx.draw_matplotlib(zx_graph, labels=True)  # We also need a matplotlib figure to overlay the original ZX-graph over the final visualisation produced by topologiq.

This circuit/graph is quite large. This notebook looks at optimisation strategies in the second section. 

However, for demonstration purposes, we will run the circuit as-is. The result should be a rather-large-yet-topologically-correct lattice surgery / space-time diagram. 

For this, we convert the zx_graph into ***topologiq's*** native graph object, a `simple_graph`, and fed this `simple_graph` to the algorithm.

> **Note.** If the PyZX graph has additional information like phases or Pauli webs, this step will momentarily lose that information. There are other methods not covered in this notebook to recover and apply this information to outputs.

In [ ]:
# CONVERT GRAPH TO TOPOLOGIQ'S NATIVE OBJECT
simple_graph = pyzx_g_to_simple_g(zx_graph)

# PARAMS & HYPERPARAMS
circuit_name = "steane_from_qiskit"
visualisation = "final"  # Calls an interactive 3D visualisation at the end. Change to None to deactivate visualisation.
animation = None  # If "GIF" or "MP4", creates a summary animation of the process (significant runtime impact).
VALUE_FUNCTION_HYPERPARAMS = (
    -1,  # Weight for lenght of path
    -1,  # Weight for number of "beams" broken by path
)

kwargs = {
    "weights": VALUE_FUNCTION_HYPERPARAMS,
    "length_of_beams": 9,
}

simple_graph_after_use, edge_pths, lattice_nodes, lattice_edges = runner(
    simple_graph,
    circuit_name,
    strip_boundaries=False,
    hide_boundaries=False,
    max_attempts=10,
    visualise=(visualisation, animation),
    fig_data=fig_data,
    **kwargs
)

## Non-descript QASM -> PyZX with optimisation

The circuit above is somewhat difficult to simplify. We invite you to give it a try as exercise. 

To keep the notebook simple and stick to demonstrating functionality, we'll now switch to a non-descript QASM file written by hand. This non-descript QASM file can be simplified in less steps than the one in the previous section, which should allow the focus to stay on the overall process rather than specific PyZX simplification strategies.

So, first, let's load the QASM file.

In [ ]:
# Thanks [Kabir Dubey](https://github.com/KabirDubey) for the QASM file and code/rationale in this block. 

qasm_file_path = str(os.path.join(root_folder, "assets/graphs", "steane_code_compressed.qasm"))

try:
    zx_circuit = zx.Circuit.from_qasm_file(qasm_file_path)
except Exception as e:
    with open(qasm_file_path, "r") as f:
        qasm_str = f.read()
    zx_circuit = zx.qasm(qasm_str)

zx_graph = zx_circuit.to_graph()

zx.draw(zx_graph, labels=True)  # PyZX produces Jupyter visualisations using D3.
fig_data = zx.draw_matplotlib(zx_graph, labels=True)  # We also need a matplotlib figure to overlay the original ZX-graph over the final visualisation produced by topologiq.

Now, let's simplify the graph a little using PyZX native functions.

In [ ]:
# Thanks [Purva Thakre](https://github.com/purva-thakre) for research into and ideas into QASM/PyZX simplifying strategies, which informed choices in this block.

# Make a copy to avoid over-writing the imported circuit.
zx_graph_optimised = zx_graph.copy()

# Auto-simplify using spider fusion method.
zx.simplify.spider_simp(zx_graph_optimised)

zx.draw(zx_graph_optimised, labels=True)  # Jupyter visualisations using D3.
fig_data = zx.draw_matplotlib(zx_graph_optimised, labels=True)  # Fig to overlay in final vis.

After, we only need to transform PyZX graph into **topologiq's** native graph format run it. 

> **Note 1.** If the PyZX graph has additional information like phases or Pauli webs, this step will momentarily lose that information. There are other methods not covered in this notebook to recover and apply this information to outputs.

> **Note 2.** Visualisations and animations are **enabled**. **Topologiq** will create a PNG for each edge completed, which takes quite a bit of time for large circuits with many edges. Accordingly, the following block of code will take some time to complete. Without visualisations and animations, the code below takes fractions of a second.

In [ ]:
# CONVERT GRAPH TO TOPOLOGIQ'S NATIVE OBJECT
simple_graph = pyzx_g_to_simple_g(zx_graph_optimised)

# PARAMS & HYPERPARAMS
circuit_name = "steane_optimised"
visualisation = "final"  # Calls an interactive 3D visualisation at the end. Change to None to deactivate visualisation.
animation = None  # If "GIF" or "MP4", creates a summary animation of the process (significant runtime impact).
VALUE_FUNCTION_HYPERPARAMS = (
    -1,  # Weight for lenght of path
    -1,  # Weight for number of "beams" broken by path
)

kwargs = {
    "weights": VALUE_FUNCTION_HYPERPARAMS,
    "length_of_beams": 9,
}

simple_graph_after_use, edge_pths, lattice_nodes, lattice_edges = runner(
    simple_graph,
    circuit_name,
    strip_boundaries=False,
    hide_boundaries=False,
    max_attempts=10,
    visualise=(visualisation, animation),
    fig_data=fig_data,
    **kwargs
)

We can simplify the circuit even more using ZX-rules manually.

In [ ]:
# Thanks [Purva Thakre](https://github.com/purva-thakre) for research into and ideas into QASM/PyZX simplifying strategies, which informed choices in this block.

# Spider-fusion method will not merge boundaries connected to single spiders.
# This is related to how PyZX presents graphs.
# But we can merge manually applying the spider-fusion rule.
for pair in [(0, 25), (1, 26), (2, 27), (3, 28)]:
    zx_graph_optimised.add_edge(zx_graph_optimised.edge(pair[0], pair[1]))
    zx_graph_optimised.remove_vertex(pair[0])

for pair in [(4, 29), (5, 30), (6, 31)]:
    zx_graph_optimised.add_edge(zx_graph_optimised.edge(pair[1], pair[0]))
    zx_graph_optimised.remove_vertex(pair[1])

zx_graph_optimised.normalize()

zx.draw(zx_graph_optimised, labels=True)  # Jupyter visualisations using D3.
fig_data = zx.draw_matplotlib(zx_graph_optimised, labels=True)  # Fig to overlay in final vis.

Let's now run this final optimised version.

In [ ]:
# CONVERT GRAPH TO TOPOLOGIQ'S NATIVE OBJECT
simple_graph = pyzx_g_to_simple_g(zx_graph_optimised)

# PARAMS & HYPERPARAMS
circuit_name = "steane_optimised"
visualisation = "final"  # Calls an interactive 3D visualisation at the end. Change to None to deactivate visualisation.
animation = None  # If "GIF" or "MP4", creates a summary animation of the process (significant runtime impact).
VALUE_FUNCTION_HYPERPARAMS = (
    -1,  # Weight for lenght of path
    -1,  # Weight for number of "beams" broken by path
)

kwargs = {
    "weights": VALUE_FUNCTION_HYPERPARAMS,
    "length_of_beams": 9,
}

simple_graph_after_use, edge_pths, lattice_nodes, lattice_edges = runner(
    simple_graph,
    circuit_name,
    strip_boundaries=False,
    hide_boundaries=False,
    max_attempts=10,
    visualise=(visualisation, animation),
    fig_data=fig_data,
    **kwargs
)

For memory's sake, let's shut down any open plots.

In [ ]:
from matplotlib.pyplot import close
close()